In [2]:
import sys
#sys.path.append('/home/osboxes/CIT_SINGALUN_Test/singalun/')
import singalun.network as sn
import pandas as pd
import numpy as np
import singalun.network as sn
import networkx as nx

In [5]:
data_T=pd.read_csv('/home/osboxes/CIT_SINGALUN_Test/GSE39582_Expression.tsv',sep='\t',header=0,index_col = 0)
#data_T.head()
#data_T.isnull().sum().sum()
#index_col = 0 consider first column as index

In [3]:
adj_pcc = sn.compute_adjacency_matrix(data_T, method = 'pearson', 
                perform_wgcna = False, verbose = True)


Correlation matrix computed (using pearson).


In [9]:
adj_wgcna= sn.compute_adjacency_matrix(data_T, method = 'pearson', 
              perform_wgcna = True, estimate_beta = True, verbose = True)

           

Correlation matrix computed (using pearson).
Adjacency matrix computed (Beta = 4).


/home/osboxes/CIT_SINGALUN_Test/singalun/network.py:120: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  model, resid = np.linalg.lstsq(reg_a, log_p_k)[:2]


In [7]:
 # We save the networks as adjacency matrices.
#adj_pcc.to_csv('/home/osboxes/CIT_SINGALUN_Test/GSE39582_correl_net.csv', sep = '\t')
adj_wgcna.to_csv('/home/osboxes/CIT_SINGALUN_Test/GSE39582_wgcna_net.csv', sep = '\t')



In [4]:
# I- We read the adjacency matrix, and create the network.
adj = pd.read_csv('/home/osboxes/CIT_SINGALUN_Test/GSE39582_correl_net.csv', sep = "\t", header = 0, 
            index_col = 0)
network = sn.create_network(adj.as_matrix(), verbose = True)
#network.nodes
nodes=list(data_T.columns)
nodes=nodes[1:]

Network created (with 585 nodes and 170820 edges).


In [56]:
#compute the centralities
# Dictionnary of all functions available for centrality.
all_functions = {'degree' : dict(nx.degree_centrality(network)),
    'weighteddegree' : dict(network.degree(weight = "weight")),
          'betweenness' :dict(nx.betweenness_centrality(network, normalized = True, 
        weight = "weight"))
,
    'cfcloseness' : dict(nx.current_flow_closeness_centrality(network, 
        weight = "weight", solver='lu'))
,
    'cfbetweenness' : dict(nx.approximate_current_flow_betweenness_centrality(
            network, normalized = True, weight = 'weight', solver = 'lu'))
,
    'eigenvector' : dict(nx.eigenvector_centrality_numpy(network, weight = "weight")),
    'katz' : dict(nx.katz_centrality_numpy(network, normalized = True, 
        weight = "weight"))           

}
for x,y in all_functions.items():
    node=nodes
    data_final=y
    df=pd.DataFrame(list(data_final.values()),index=node,columns=[x])
    df.to_csv('GSE39582_'+x+'.tsv',header = True, index = True,sep='\t')    
        

In [ ]:
import os
df = pd.DataFrame()
for file in os.listdir('Centralities/'):
    df.concat([df, pd.read_csv('Centralities/' + file, dtype = 'float')], axis = 1)
df.to_csv('Centralities/GSE39582_centr.csv')

In [7]:
all_functions = {'PageRank' : dict(nx.pagerank(network, weight = "weight"))}
for x,y in all_functions.items():
    node=nodes
    data_final=y
    df=pd.DataFrame(list(data_final.values()),index=node,columns=[x])
    df.to_csv('GSE39582_'+x+'.tsv',header = True, index = True,sep='\t')  

In [20]:
hubs,authorities= nx.hits(network, normalized = True)

0.0017082314332620147
0.0017060833130446607
0.0017040672395677611
0.0017134375371130313
0.0017044903966283523
0.0016973300221369253
0.0016990290051069376
0.0017265378990346171
0.0017303012937117355
0.0017116489894742292
0.001704548959372164
0.0016898785589420132
0.0016998861337423766
0.001699045159779415
0.0017226000813246696
0.0017159878020532899
0.0016799580872612493
0.0017209367795885415
0.0017084975606209264
0.0016888816085487736
0.001723314781884597
0.0017186517655647007
0.0017071542578622814
0.0017071714064993489
0.0017086245668594268
0.0017101966482105263
0.0017244524643335063
0.0017102619813679938
0.001717450026274273
0.0016438473939410746
0.0016994434827966478
0.001696864021796658
0.0017127752736751762
0.0017224692320980607
0.001709271823932273
0.0017169392595455272
0.0017272537393273953
0.0017161784149157119
0.0017202661995673744
0.0016931912932265463
0.0017057452679146317
0.0017216803278577966
0.0017319382624739071
0.0016996014957465864
0.0017290238719060006
0.00170590208160

In [ ]:
S = nx.to_scipy_sparse_matrix(Graph, nodelist=nodelist)
new=open('/home/osboxes/sparse/'+fname+'.tsv','w+')
Final=S.A
for (v,y),val in np.ndenumerate(Final):
           new.write('%s%s%s%s%s%s' %(nodelist[v],'\t',val,'\t',nodelist[y],'\n'))
new.close()